# Imports

In [1]:
# Native python
import os

In [14]:
# External
import pandas as pd
import sklearn.model_selection

In [3]:
# Custom scripts
from nitelite_mapmaker import mapmaker, georeference

# Settings

In [17]:
settings = dict(
    # Data architecture
    flight_name = '220513-FH135',
    data_dir = '/Users/Shared/data/nitelite',
    image_dir = '/Users/Shared/data/nitelite/images',
    google_drive_dir = '/Users/zhafensaavedra/Google Drive/Shared drives/NITELite/Data & Analysis',
    flight_subdir = 'Old NITELite Flights/220513-FH135',
    reffed_subdir = 'QGIS FH135/FH135 Main Project/Main Geo Files',
    img_log_filename = 'image.log',
    imu_log_filename = 'OBC/PresIMULog.csv',
    gps_log_filename = 'OBC/GPSLog.csv',
    
    # Choices for what to process
    camera_num = 1,
    test_size = 0.2,
)

# Set Up

## Settings Parsing

In [18]:
# The camera has an according long number
settings['camera_long_num'] = settings['camera_num'] + 23085686

In [19]:
# Data architecture processing
settings['image_dir'] = os.path.join(
    settings['data_dir'],
    'nitelite/images',
    settings['flight_name'],
    str(settings['camera_long_num'])
)
settings['metadata_dir'] = os.path.join(
    settings['google_drive_dir'],
    settings['flight_subdir'],
    'data',
)
settings['reffed_dir'] = os.path.join(
    settings['google_drive_dir'],
    settings['reffed_subdir'],
)

## Object Creation and Preprocessing

In [20]:
# Create the main mapmaker object
mm = mapmaker.Mapmaker(
    image_dir=settings['image_dir'],
    img_log_fp=os.path.join(settings['metadata_dir'], settings['img_log_filename']),
    imu_log_fp=os.path.join(settings['metadata_dir'], settings['imu_log_filename']),
    gps_log_fp=os.path.join(settings['metadata_dir'], settings['gps_log_filename']),
)

In [21]:
# General metadata loading
mm.prep()

In [22]:
# Manually-georeferenced metadata
_ = mm.flight.get_manually_georeferenced_filepaths(
    settings['reffed_dir']
)

# Train-test Split

In [23]:
# Get data
reffed = mm.flight.metadata[mm.flight.metadata['manually_referenced_fp'].notna()]

In [27]:
# Perform the split
train_inds, test_inds = sklearn.model_selection.train_test_split(
    reffed.index,
    test_size=settings['test_size'],
)
reffed_train = reffed.loc[train_inds]
reffed_test = reffed.loc[test_inds]

# Mapmake

The process will be:
1. Split the mosaick into tiles
2. Mosaick all referenced images for a tile
3. ???

# Validate Map